# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [ ]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [ ]:
df_eng = pd.read_csv("data/movie_reviews_eng.csv")

In [ ]:
df_eng.sample(10)

In [ ]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original_Language'] = 'English'
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original_Language'] = 'French'
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp['Original_Language'] = 'Spanish'
    
    #Ensures that all DataFrames have the same column names
    df_fr.columns = df_eng.columns
    df_sp.columns = df_eng.columns
    
    #Combines the three DataFrames (df_eng, df_French, and df_Spanish) into a single DataFrame named df.
    #Resets the index of the combined DataFrame to avoid issues with duplicate indices.
    df = pd.concat([df_eng, df_fr, df_sp]).reset_index(drop='True')
    
    df.rename(columns={'Movie / TV Series': 'Title'}, inplace=True)
    return df

df = preprocess_data()

In [ ]:
df.sample(10)

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [ ]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianMTModel.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [ ]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews =  df[df.Original_Language == 'French']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df.Original_Language == 'French']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df.Original_Language == 'Spanish']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df.Original_Language == 'Spanish']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

df.loc[df.Original_Language == 'French', 'Review'] = fr_reviews_en
df.loc[df.Original_Language == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df.Original_Language == 'Spanish', 'Review'] = es_reviews_en
df.loc[df.Original_Language == 'Spanish', 'Synopsis'] = es_synopsis_en

In [ ]:
df.sample(10)

### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [ ]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)[0]
    sentiment = "Positive" if result['label'] == 'POSITIVE' else "Negative"

    return sentiment

In [ ]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))


In [ ]:
df.sample(10)

In [ ]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)